In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(torch.cuda.get_arch_list())

print(f"Is CUDA supported by this system?{torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
       
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
import torch
torch.cuda.empty_cache()

In [12]:
import onnx
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import numpy as np
import matplotlib.pyplot as plt
import itertools
import os
from PIL import Image
from torch import optim
from tqdm import tqdm
import netron
import time
from IPython.display import IFrame

def show_netron(model_path, port):
    time.sleep(3.)
    netron.start(model_path, address=("localhost", port), browse=False)
    return IFrame(src=f"http://localhost:{port}/", width="100%", height=400)


In [22]:
import torch.nn as nn
import torch.nn.functional as F
import brevitas.nn as qnn
import numpy as np

class ECGNet(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        myWeight_bit_width = 2
        self.features = nn.Sequential(
            qnn.QuantIdentity(),

            qnn.QuantConv2d(1, 16, kernel_size=(1, 5),
                            weight_bit_width=myWeight_bit_width),
            qnn.QuantReLU(inplace=True, weight_bit_width=myWeight_bit_width),
            qnn.QuantConv2d(16, 32, kernel_size=(1, 5),
                            weight_bit_width=myWeight_bit_width),
            qnn.QuantReLU(inplace=True, weight_bit_width=myWeight_bit_width),
            nn.MaxPool2d(kernel_size=(1, 4), stride=(1, 4)),

            qnn.QuantConv2d(32, 16, kernel_size=(1, 3),
                            weight_bit_width=myWeight_bit_width),
            qnn.QuantReLU(inplace=True, weight_bit_width=myWeight_bit_width),
            qnn.QuantConv2d(16, 16, kernel_size=(1, 3),
                            weight_bit_width=myWeight_bit_width),
            qnn.QuantReLU(inplace=True, weight_bit_width=myWeight_bit_width),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
        )

        self.classifier = nn.Sequential(
            qnn.QuantLinear(16 * 13, num_classes, bias=True,
                            weight_bit_width=myWeight_bit_width),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.reshape(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "trained_models/ecgnet_w2a2_model.pth"
model = ECGNet()
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

<All keys matched successfully>

In [24]:
from brevitas.core.quant import QuantType
from brevitas.nn import QuantIdentity


class ECGNetForExport(nn.Module):
    def __init__(self, my_pretrained_model):
        super(ECGNetForExport, self).__init__()
#         self.qnt_input = QuantIdentity(quant_type=QuantType.FP, bit_width=32)
        self.pretrained = my_pretrained_model
        self.qnt_output = QuantIdentity(quant_type=QuantType.INT, bit_width=8)
    
    def forward(self, x):
        # assume x contains bipolar {-1,1} elems
        # shift from {-1,1} -> {0,1} since that is the
        # input range for the trained network
#         x = (x + torch.tensor([1.0])) / 2.0  
        out_original = self.pretrained(x)
        out_final = self.qnt_output(out_original)   # output as {-1,1}     
        return out_final

model_for_export = ECGNetForExport(model)

In [25]:
import onnx
import torch
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup

from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
from qonnx.core.datatype import IntType
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from  brevitas.quant_tensor import QuantTensor

export_onnx_path = 'onnx_models/ecgnet_w2a2_export.onnx'
input_shape = (1, 1, 1, 128)
input_a = np.random.randint(0,1,size=input_shape).astype(np.float32)
#input_a = 2*input_a - 1
scale = 1.0
input_t = torch.from_numpy(input_a * scale)

export_qonnx(model_for_export,export_path = export_onnx_path,input_t = input_t)
qonnx_cleanup(export_onnx_path,out_file = export_onnx_path)

model_for_export = ModelWrapper('onnx_models/ecgnet_w2a2_export.onnx')
model_for_export.set_tensor_datatype(model_for_export.graph.input[0].name, DataType["INT8"])
model_for_export = model_for_export.transform(ConvertQONNXtoFINN())
model_for_export.save(export_onnx_path)


/home/aditya/finn/deps/qonnx/src/qonnx/transformation/gemm_to_matmul.py:57: UserWarning: The GemmToMatMul transformation only offers explicit support for version 9 of the Gemm node, but the ONNX version of the supplied model is 14. Thus the transformation may fail or return incomplete results.
  warnings.warn(


In [ ]:
from finn.util.visualization import showInNetron
showInNetron(export_onnx_path)

In [26]:
pred = model(input_t)
print(pred)

tensor([[ 3.6016, -3.4684]], grad_fn=<AddmmBackward0>)


In [27]:
data = np.load("datasets/last_val_data/0/101_396.npy")
data2 = np.load("datasets/last_val_data/1/118_6057.npy")

tensor_data = torch.from_numpy(data)
tensor_data = tensor_data.unsqueeze(0)  
tensor_data2 = torch.from_numpy(data2)
tensor_data2 = tensor_data2.unsqueeze(0)  

pred1 = model(tensor_data)
pred2 = model(tensor_data2)

print(pred1)
print(pred2)

tensor([[ 2.7631, -2.5251]], grad_fn=<AddmmBackward0>)
tensor([[-4.3289,  5.2307]], grad_fn=<AddmmBackward0>)


In [28]:
from qonnx.core.datatype import DataType

model_for_sim = ModelWrapper('onnx_models/ecgnet_w2a2_export.onnx')
finnonnx_in_tensor_name = model_for_sim.graph.input[0].name
finnonnx_out_tensor_name = model_for_sim.graph.output[0].name
print("Input tensor name: %s" % finnonnx_in_tensor_name)
print("Output tensor name: %s" % finnonnx_out_tensor_name)
finnonnx_model_in_shape = model_for_sim.get_tensor_shape(finnonnx_in_tensor_name)
finnonnx_model_out_shape = model_for_sim.get_tensor_shape(finnonnx_out_tensor_name)
print("Input tensor shape: %s" % str(finnonnx_model_in_shape))
print("Output tensor shape: %s" % str(finnonnx_model_out_shape))
finnonnx_model_in_dt = model_for_sim.get_tensor_datatype(finnonnx_in_tensor_name)
finnonnx_model_out_dt = model_for_sim.get_tensor_datatype(finnonnx_out_tensor_name)
print("Input tensor datatype: %s" % str(finnonnx_model_in_dt.name))
print("Output tensor datatype: %s" % str(finnonnx_model_out_dt.name))
print("List of node operator types in the graph: ")
print([x.op_type for x in model_for_sim.graph.node])

Input tensor name: global_in
Output tensor name: global_out
Input tensor shape: [1, 1, 1, 128]
Output tensor shape: [1, 2]
Input tensor datatype: INT8
Output tensor datatype: FLOAT32
List of node operator types in the graph: 
['MultiThreshold', 'Add', 'Mul', 'Conv', 'Mul', 'Add', 'MultiThreshold', 'Mul', 'Conv', 'Mul', 'Add', 'MultiThreshold', 'Mul', 'MaxPool', 'Conv', 'Mul', 'Add', 'MultiThreshold', 'Mul', 'Conv', 'Mul', 'Add', 'MultiThreshold', 'Mul', 'MaxPool', 'Reshape', 'MatMul', 'Mul', 'Add', 'MultiThreshold', 'Add', 'Mul']


In [19]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors

model_for_sim = model_for_sim.transform(InferShapes())
model_for_sim = model_for_sim.transform(FoldConstants())
model_for_sim = model_for_sim.transform(GiveUniqueNodeNames())
model_for_sim = model_for_sim.transform(GiveReadableTensorNames())
model_for_sim = model_for_sim.transform(InferDataTypes())
model_for_sim = model_for_sim.transform(RemoveStaticGraphInputs())
model_for_sim = model_for_sim.transform(Change3DTo4DTensors())

verif_model_filename = 'onnx_models/ecgnet_w2a2_export_verif.onnx'
model_for_sim.save(verif_model_filename)

In [ ]:
import torch
import numpy as np
from finn.util.test import get_topk
from finn.util.pytorch import ToTensor

model = ECGNet()  

input_size = (1,1,1,128)
input_tensor_npy = np.random.randint(0,1,input_size)  

input_tensor_torch = torch.from_numpy(input_tensor_npy).float()

input_tensor_torch = ToTensor().forward(input_tensor_torch).detach().numpy()

with torch.no_grad():
    model.eval()  
    output_tensor_npy = model(input_tensor_torch).numpy()

output_tensor_npy  = get_topk(output_tensor_npy,k = 1)

np.save('input.npy', input_tensor_npy)
np.save('expected_output.npy', output_tensor_npy)


In [ ]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "onnx_models/ecgnet_w2a2_export.onnx"
output_dir = "output_verification_with_estimates/"

#Delete previous run results if exist
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print("Previous run results deleted!")

cfg_estimates = build.DataflowBuildConfig(
    output_dir          = output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 10000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7z020clg400-1",
    board               = "Pynq-Z2",
    steps               = build_cfg.estimate_only_dataflow_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ],
    verify_steps=[
        build_cfg.VerificationStepType.QONNX_TO_FINN_PYTHON,
        build_cfg.VerificationStepType.TIDY_UP_PYTHON,
        build_cfg.VerificationStepType.STREAMLINED_PYTHON,
    ]
)

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg_estimates);